### **Apache Beam**

In this notebook, I have set up Apache beam to perform operations such as:
 - Map, filter and core transforms
 - ParDo transform
 - Composite transform
 
The data set consists of shopping data of customers who shopped IT related products. We perform operations on these customers to group customers who bought specific category Items, customers who return to buy the same product and customers who dont.

Installing Apache beam

In [ ]:
!pip install apache-beam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.9 MB 3.9 MB/s 
     |████████████████████████████████| 508 kB 50.2 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 151 kB 55.2 MB/s 
     |████████████████████████████████| 2.4 MB 62.9 MB/s 
     |████████████████████████████████| 47 kB 4.2 MB/s 
     |████████████████████████████████| 270 kB 57.9 MB/s 
     |████████████████████████████████| 1.0 MB 65.9 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=9db679685b7a475345ae0dd95970a0e1a4cd06de8992cb1b1dc276271948756c
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=9bc09c4a2b6e1faefbff7145f6232e8aebcd23294e2e7c572118dc3e09e38dae
  Stored in directory: /root/.ca

Get the input shopping data of customers, calculate number of customers who bought monitor. Write their details into an output file. Core transform- map, filter is used here, which is done on the basis of customerID. 

In [ ]:
import apache_beam as beam

pipe1 = beam.Pipeline()

shopping_count = (
    
    pipe1
    |beam.io.ReadFromText('/content/sample_data/Shopping_Input_Apache.txt')

    |beam.Map(lambda record: record.split(","))
    |beam.Filter(lambda record: record[2] == 'Monitor')
    |beam.Map(lambda record: (record[1],1))
    |beam.CombinePerKey(sum)

    |beam.io.WriteToText('output_MonitorSales_count.txt')
)

pipe1.run()

Filter and calculate the number of customers who bought headphones using ParDo transform, adn write result to an output file

In [ ]:
import apache_beam as beam

class SplitRow(beam.DoFn):
 
  def process(self, element):
    return  [element.split(',')]
 

class FilterHeadphoneCustomers(beam.DoFn):
 
  def process(self, element):
    if element[2] == 'Headphones':
      return [element]  
    
class PairCustomers(beam.DoFn):
 
  def process(self, element):
    return [(element[1], 1)]
 
class Counting(beam.DoFn):
 
  def process(self, element):
    (key, values) = element         
    return [(key, sum(values))]
     

pipe2 = beam.Pipeline()

sales_count = (
    
   pipe2
    |beam.io.ReadFromText('/content/sample_data/Shopping_Input_Apache.txt')
    
    |beam.ParDo(SplitRow())
    |beam.ParDo(FilterHeadphoneCustomers())
    |beam.ParDo(PairCustomers())
    | 'Group ' >> beam.GroupByKey()
    | 'Sum using ParDo' >> beam.ParDo(Counting())  
    
    |beam.io.WriteToText('pardo_output_headphones_sales.txt')
 
)

pipe2.run()

Using Composite transform to group and calculate all(returning and non-returning) customers of every item in the input file such as chair, monitor, mouse, headphones and keyboard - and write the details to an output file.

In [ ]:
import apache_beam as beam

class CustomTransform(beam.PTransform):
  
  def expand(self, input_coll):
    
    a = ( 
        input_coll
                       | 'Group and sum' >> beam.CombinePerKey(sum)
                       | 'Customers and sales' >> beam.Map(format_output)
              
    )
    return a

def SplitRow(element):
    return element.split(',')
  
def format_output(element):
  name, count = element
  return ', '.join((name,str(count)))

pipe3 = beam.Pipeline()

input_collection = ( 
                      pipe3 
                      | "Read from text file" >> beam.io.ReadFromText('/content/sample_data/Shopping_Input_Apache.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

monitor_customer_sales_count = (
                      input_collection
                      | 'Get all cusotmer who shopped Monitors' >> beam.Filter(lambda record: record[2] == 'Monitor')
                      | 'Pair each Monitor customer with 1' >> beam.Map(lambda record: ("Monitor, " +record[1], 1))
                      | 'composite Monitor customers' >> CustomTransform()
                      | 'Write results for Monitor' >> beam.io.WriteToText('monitor_output.txt')
                 )

keyboard_customer_sales_count = (
                input_collection
                | 'Get all customers who shopped keyboards' >> beam.Filter(lambda record: record[2] == 'Keyboard')
                | 'Pair each Keyboard customer with 1' >> beam.Map(lambda record: ("Keyboard, " +record[1], 1))
                | 'composite Keyboard customers' >> CustomTransform()
                | 'Write results for Keyboard' >> beam.io.WriteToText('keyboard_output.txt')
           )

headphones_customer_sales_count = (
                      input_collection
                      | 'Get all cusotmer who shopped Headphones' >> beam.Filter(lambda record: record[2] == 'Headphones')
                      | 'Pair each headphone customer with 1' >> beam.Map(lambda record: ("Headphones, " +record[1], 1))
                      | 'composite Headphones customers' >> CustomTransform()
                      | 'Write results for Headphones' >> beam.io.WriteToText('headphones_output.txt')
                 )

mouse_customer_sales_count = (
                input_collection
                | 'Get all customers who shopped mouse' >> beam.Filter(lambda record: record[2] == 'Mouse')
                | 'Pair each mouse customer with 1' >> beam.Map(lambda record: ("Mouse, " +record[1], 1))
                | 'composite Keyboard mouse' >> CustomTransform()
                | 'Write results for mouse' >> beam.io.WriteToText('mouse_output.txt')
                )

chair_customer_sales_count = (
                      input_collection
                      | 'Get all cusotmer who shopped chairs' >> beam.Filter(lambda record: record[2] == 'Chair')
                      | 'Pair each chair customer with 1' >> beam.Map(lambda record: ("Chair, " +record[1], 1))
                      | 'composite chair customers' >> CustomTransform()
                      | 'Write results for chair' >> beam.io.WriteToText('chair_output.txt')
                 )

pipe3.run()

Using Composite transform to group, filter and identify returning customers of every item in the input file such as chair, monitor, mouse, headphones and keyboard - and write the details to an output file.

In [ ]:
import apache_beam as beam

class CustomTransform(beam.PTransform):
  
  def expand(self, input_coll):
    
    a = ( 
        input_coll
                       | 'Group and sum' >> beam.CombinePerKey(sum)
                       | 'count filter accounts' >> beam.Filter(filter_on_count)
                       | 'Returning Customers' >> beam.Map(format_output)
              
    )
    return a

def SplitRow(element):
    return element.split(',')

def filter_on_count(element):
  name, count = element
  if count > 1:
    return element
  
def format_output(element):
  name, count = element
  return ', '.join((name,str(count), 'Returning Customer'))

pipe4 = beam.Pipeline()

input_collection = ( 
                      pipe4
                      | "Read from text file" >> beam.io.ReadFromText('/content/sample_data/Shopping_Input_Apache.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

monitor_customer_sales_count = (
                      input_collection
                      | 'Get all cusotmer who shopped Monitors' >> beam.Filter(lambda record: record[2] == 'Monitor')
                      | 'Pair each Monitor customer with 1' >> beam.Map(lambda record: ("Monitor, " +record[1], 1))
                      | 'composite Monitor customers' >> CustomTransform()
                      | 'Write results for Monitor' >> beam.io.WriteToText('regular_cust_monitor_output.txt')
                 )

keyboard_customer_sales_count = (
                input_collection
                | 'Get all customers who shopped keyboards' >> beam.Filter(lambda record: record[2] == 'Keyboard')
                | 'Pair each Keyboard customer with 1' >> beam.Map(lambda record: ("Keyboard, " +record[1], 1))
                | 'composite Keyboard customers' >> CustomTransform()
                | 'Write results for Keyboard' >> beam.io.WriteToText('regular_cust_keyboard_output.txt')
           )

headphones_customer_sales_count = (
                      input_collection
                      | 'Get all cusotmer who shopped Headphones' >> beam.Filter(lambda record: record[2] == 'Headphones')
                      | 'Pair each headphone customer with 1' >> beam.Map(lambda record: ("Headphones, " +record[1], 1))
                      | 'composite Headphones customers' >> CustomTransform()
                      | 'Write results for Headphones' >> beam.io.WriteToText('regular_cust_headphones_output.txt')
                 )

mouse_customer_sales_count = (
                input_collection
                | 'Get all customers who shopped mouse' >> beam.Filter(lambda record: record[2] == 'Mouse')
                | 'Pair each mouse customer with 1' >> beam.Map(lambda record: ("Mouse, " +record[1], 1))
                | 'composite Keyboard mouse' >> CustomTransform()
                | 'Write results for mouse' >> beam.io.WriteToText('regular_cust_mouse_output.txt')
                )

chair_customer_sales_count = (
                      input_collection
                      | 'Get all cusotmer who shopped chairs' >> beam.Filter(lambda record: record[2] == 'Chair')
                      | 'Pair each chair customer with 1' >> beam.Map(lambda record: ("Chair, " +record[1], 1))
                      | 'composite chair customers' >> CustomTransform()
                      | 'Write results for chair' >> beam.io.WriteToText('regular_cust_chair_output.txt')
                 )

pipe4.run()